# NSD Final Project

## Title: Reinforcement Learning in Options Hedging 

### Team member: Jiang, T., Cui, D.F. and Wang, Y.Z.

$$
\newcommand{\supp}{\mathrm{supp}}
\newcommand{\E}{\mathbb{E} }
\newcommand{\Eof}[1]{\mathbb{E}\left[ #1 \right]}
\def\Cov{{ \mbox{Cov} }}
\def\Var{{ \mbox{Var} }}
\newcommand{\1}{\mathbf{1} }
\newcommand{\PP}{\mathbb{P} }
\newcommand{\Pof}[1]{\mathbb{P}\left[ #1 \right]}
%\newcommand{\Pr}{\mathrm{Pr} }
\newcommand{\QQ}{\mathbb{Q} }
\newcommand{\RR}{\mathbb{R} }
\newcommand{\DD}{\mathbb{D} }
\newcommand{\HH}{\mathbb{H} }
\newcommand{\spn}{\mathrm{span} }
\newcommand{\cov}{\mathrm{cov} }
\newcommand{\sgn}{\mathrm{sgn} }
\newcommand{\HS}{\mathcal{L}_{\mathrm{HS}} }
%\newcommand{\HS}{\mathrm{HS} }
\newcommand{\trace}{\mathrm{trace} }
\newcommand{\LL}{\mathcal{L} }
%\newcommand{\LL}{\mathrm{L} }
\newcommand{\s}{\mathcal{S} }
\newcommand{\ee}{\mathcal{E} }
\newcommand{\ff}{\mathcal{F} }
\newcommand{\hh}{\mathcal{H} }
\newcommand{\bb}{\mathcal{B} }
\newcommand{\dd}{\mathcal{D} }
\newcommand{\g}{\mathcal{G} }
\newcommand{\p}{\partial}
\newcommand{\half}{\frac{1}{2} }
\newcommand{\T}{\mathcal{T} }
\newcommand{\bi}{\begin{itemize}}
\newcommand{\ei}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\beas}{\begin{eqnarray*}}
\newcommand{\eeas}{\end{eqnarray*}}
\newcommand{\cO}{\mathcal{O}}
\newcommand{\cF}{\mathcal{F}}
\newcommand{\cL}{\mathcal{L}}
\newcommand{\BS}{\text{BS}}
$$

### Abstract

### Table of content
- Introduction
    * Delta Hedging
    * Reinforcement Learning
- Toolkits
    * Gym
    * Spinning Up
    * Installation
- Our Setting
    * Learning Algorithms 
    * Models
    * Formulation of The Problem
    * The Implementation of Environments
- Experiment
    * Detailed Setting
        - Parameters of Models
        - Parameters of Environments
    * The Results of Our Method
    * The Results of Delta Hedging
    * Comparison
- Conclusion and Discussions
    

### Introduction

### Toolkits
#### Gym
The following content is from gym website: http://gym.openai.com
- ' Gym is a toolkit for developing and comparing reinforcement learning algorithms. It makes no assumptions about the structure of your agent, and is compatible with any numerical computation library, such as TensorFlow or Theano.'
-----------
we use gym to implement our own environment, the environment code is also below, yet we need to do the following installation to follow the gym style but not simply define the function in the jupyter notebook

#### Spinning up
The following content is from Spinning up website: https://spinningup.openai.com
- '...So our package here is designed to serve as the missing middle step for people who are excited by deep RL, and would like to learn how to use it or make a contribution, but don’t have a clear sense of what to study or how to transmute algorithms into code. We’ve tried to make this as helpful a launching point as possible.'
----------------------
we use spinningup to implement our algorithms, but we change their code, so we enclose this packages in the work dir.

#### Installation
The installation of spinup may take some time because it have many related packages, if you don't want to train the model yourself but just run our testing code, you at least need to have pytorch.

In [ ]:
!pip3 install torch torchvision#if you don't want to install spinup

In [ ]:
!pip3 install -e spinup#for spinup

In [ ]:
!pip3 install -e gym#for gym

In [ ]:
!pip3 install -e DHENV#install our own event

### Our Setting
#### Models
Because ddpg is chosen as our algorithm, we need to model two functions. 
- The first function is the Q function, which is to map the action and state into the revenue
- and the second is the P function, which is to map the state to the action. 
* Specifically, we all choose multiple perceptrons, the simplest deep learning network, as our modeling function. On the number of layers, we choose 4 (stock number, account deposit, stock price, maturity time) / 5 (plus actions) - > 80-- > 40 -- > 1, and add leakyrelu activation layer in the middle.Because ddpg is chosen as our algorithm, we need to model two functions. The first function is the Q function, which is to map the action and state into the revenue, and the second is the P function, which is to map the state to the action. Specifically, we all choose multiple perceptrons, the simplest deep learning network, as our modeling function. On the number of layers, we choose 4 (stock number, account deposit, stock price, maturity time) / 5 (plus actions) - > 80-- > 40 -- > 1, and add leakyrelu activation layer in the middle.**In particular, we add a tanh function at the end of P function. We will talk about its importance below. **The code is as follows

In [2]:
import torch
import torch.nn as nn
import numpy as np
class HEDGING_Actor(nn.Module):

    def __init__(self, obs_dim, act_dim, act_limit):
        super().__init__()
        self.fc1 = nn.Linear(obs_dim, 80)
        self.ac1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(256, 128)
        self.ac2 = nn.LeakyReLU(0.1)
        self.fc3 = nn.Linear(80, 40)
        self.ac3 = nn.LeakyReLU(0.1)
        self.fc4 = nn.Linear(40, act_dim)
        self.th = nn.Tanh()

    def forward(self, obs):
        # Return output from network scaled to action space limits.
        c = obs.numpy()
        if np.isnan(c).any():
            print('nan')
            exit()
        for item in c:
            try:
                for it in item:
                    if np.inf == abs(it):
                        print('inf')
                        print(c)
                        exit()
            except:
                 if np.inf == abs(item):
                     print('inf')
                     print(c)
                     exit()
        x = self.fc1(obs)
        x = self.ac1(x)
        '''
        x = self.fc2(x)
        x = self.ac2(x)
        '''
        x = self.fc3(x)
        x = self.ac3(x)
        x = self.fc4(x)
        d = x.detach().numpy()
        if np.isnan(d).any():
            print('nan')
            exit()
        #x = torch.clamp(x, min = -5, max = 10)
        return self.th(x)

class HEDGING_QFunction(nn.Module):

    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.fc1 = nn.Linear(obs_dim + act_dim, 80)
        self.ac1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(256, 128)
        self.ac2 = nn.LeakyReLU(0.1)
        self.fc3 = nn.Linear(80, 40)
        self.ac3 = nn.LeakyReLU(0.1)
        self.fc4 = nn.Linear(40, act_dim)

    def forward(self, obs, act):
        x = self.fc1(torch.cat([obs, act], dim=-1))
        x = self.ac1(x)
        '''
        x = self.fc2(x)
        x = self.ac2(x)
        '''
        x = self.fc3(x)
        x = self.ac3(x)
        x = self.fc4(x)
        return torch.squeeze(x, -1) # Critical to ensure q has right shape.

class HEDGING(nn.Module):

    def __init__(self, observation_space, action_space,activation=nn.ReLU):
        super().__init__()

        obs_dim = observation_space.shape[0]
        act_dim = action_space.shape[0]
        act_limit = action_space.high[0]

        # build policy and value functions
        self.pi = HEDGING_Actor(obs_dim, act_dim, act_limit)
        self.q = HEDGING_QFunction(obs_dim, act_dim)

    def act(self, obs):
        with torch.no_grad():
            return self.pi(obs).numpy()

The reasons why the model is like that is below:
- The number of layers of the model and the number of neurons: These are the results of experiment evaluated by convergence and rewards. In the code, you can see some traces of experiments that have not been deleted. Of course, due to the limitation of time, we did not make a more detailed parameter adjustment. 
- The tanh function: This function aim to solve the problem of the infinity action and reward. The function of tanh is - 1 to 1, which makes the range of action very stable. If the tanh function is not added, it is easy to have infinity, which will cause nan to appear in the training parameters.
- Why tanh but not other: Your next question may be why you don't use other methods to solve the above problem. For example, if the action is forced to be limited to a certain range, this method will prevent the function from gradient descent. Intuitively, you can understand this thing: suppose the output value is 10000, However, we limit the action to - 5 to 5. At this time, the gradient around 10000 is 0, because whether we change 10000 into 9000 or 11000, the final benefit is the benefit with action of 5, so this method is not acceptable. Compared with the activation function of sigmoid, the range of sigmoid is 0 to 1, and tanh is from - 1 to + 1, which is more suitable for the problem of positive and negative.

### Experiment
#### Training
Training take about 7 hours, so if you do not have time do not run the cell below or you can just run to test if it run well. , and can load the network straightly from the dir.

In [ ]:
from spinup import ddpg_pytorch as ddpg#run it freely

In [ ]:
actat, ac = ddpg(env_fn=lambda : gym.make('DHENV:DHENV-v1'),pi_lr = 1e-4, q_lr = 1e-4, actor_critic = HEDGING, num_test_episodes=100)#run it to train

In [ ]:
actat = torch.load('actar.pkl')
ac = torch.load('ac.pkl')#run it to load

You may ask why there are two networks here. This is because in the ddpg algorithm, there is a training network and a final network. Because the final network is only partially updated with the parameters of the training network, it receives the initial difference model with little influence, but there is less training. Let's take a look at their performance separately.

#### Evaluation
Next, we use Monte Carlo simulation to conduct 10000 rounds of simulation tests on the two networks and delta hedging strategy respectively. Through these data, we evaluate their behavior from the two dimensions of discounted profit & loss and the final accounts value. Finally, we show their different behavior logic and try to explain the different values，**Monte Carlo simulation takes about an hour. If you don't have enough time, you can also use the data we have stored directly.**

In [1]:
import gym
import copy

In [ ]:
#run simulation
Accounts = {}
Rewards = {}
actions = {}
Prices = {}
testlist = ['ac','actar','delta']
for item in testlist:
    Accounts[item] = []
    actions[item] = []
    Rewards[item] = []
    Prices[item] = []
for item in testlist:
    if item == 'ac':
        actitem = ac
        env = gym.make('DHENV:DHENV-v1')
    elif item == 'actar':
        actitem = actat
        env = gym.make('DHENV:DHENV-v1')
    else:
        actitem = ac
        env = gym.make('DHENV:DHENV-v3')
    for j in range(10000):
        print(j)
        state = env.reset()
        act = actitem.act(torch.from_numpy(state))
        for i in range(200):
            s, r, d, _ = env.step(act)
            if d:
                break
            act = actitem.act(torch.from_numpy(s))
        Accounts[item].append(copy.deepcopy(env.Accounts))
        actions[item].append(copy.deepcopy(env.actions))
        Rewards[item].append(copy.deepcopy(env.rewards))
        Prices[item].append(copy.deepcopy(env.prices))

In [4]:
#load data
with open('Accounts.pkl','rb') as f:
    Accounts = pickle.load(f)
with open('Prices.pkl','rb') as f:
    Prices = pickle.load(f)
with open('Rewards.pkl','rb') as f:
    Rewards = pickle.load(f)
with open('actions.pkl','rb') as f:
    actions = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'Accounts.pkl'

In [ ]:
import torch
actat = torch.load('actar.pkl')
actat = torch.load('actar.pkl')

### Conclusion and Discussions

### Reference